# Financial Analysis
## (using multi-agent collaboration system)

In this project, we will use 4 agents

- Data Analyst Agent
- Trading Strategy Agent
- Execution Agent
- Risk Management Agent



Using the 4 agents, we work towards 4 tasks

- Data Analysis Task (for Data Analyst)
- Strategy Development Task (for Trading Strategy agent)
- Execution Planning Task (for Execution Agent)
-  Risk Assessment Task (for Risk Management Agent)

In this project, we will use a **Hierarchial Process**, where crewAI creates a manager to delegate task to all the 4 agents

## Step 01. Installing crewai and crewai_tools libraries

**Note:** crewai requires Python version >=3.10 and <=3.13

In [1]:
import sys
print(sys.version)

3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:46:00) [Clang 18.1.8 ]


In [2]:
!pip install crewai

In [3]:
!pip install crewai_tools

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing modules, LLMs, tools

**NOTE:** To create Agent class objects, we need to specify OPEN_AI model name and api key as environment variables

**NOTE:** To run crewai's SerperDevTool, we need SERPER_API_KEY as environment variables

In [5]:
from crewai import Crew, Agent, Task

import os
os.environ['OPENAI_API_KEY'] = ''
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ['SERPER_API_KEY'] = ''

### crewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/eolanrew/miniconda3/envs/code-demo-env/envs/exercise/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Step 03. Creating Agents

### Agent 1: Data Analyst Agent

In [7]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    allow_delegation=True,
    verbose=True,
    tools=[search_tool, scrape_tool]
)

### Agent 2: Trading Strategy Agent

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 3: Execution Agent

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 4: Risk Management Agent

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

## Step 04. Creating Tasks

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Step 06. Creating Crew

### Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

In [16]:
# define the crew with agent and tasks
financial_trading_crew = Crew (
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    verbose=False,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
)

### Running the crew

In [17]:
# specify the input for crew
financial_trading_inputs = {
    'stock_selection': 'VOO',
    'initial_capital': '100000',
    'risk_tolerance': 'Easy',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [18]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (VOO) using statistical modeling     │
│  and machine learning techniques to identify trends and predict market movements.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-07-05 13:53:21,706 - 8439663616 - serper_dev_tool.py-serper_dev_tool:182 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
2025-07-05 13:53:21,838 - 8439663616 - serper_dev_tool.py-serper_dev_tool:182 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
2025-07-05 13:53:21,998 - 8439663616 - serper_dev_tool.py-serper_dev_tool:182 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
2025-07-05 13:53:22,141 - 8439663616 - serper_dev_tool.py-serper_dev_tool:182 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message"

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Thought: Thought: To perform continuous monitoring and analysis of the VOO stock, I first need to gather the   │
│  latest market data and significant news updates regarding VOO. This will allow me to identify any current      │
│  trends or immediate changes in the stock's performance.                                                        │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"VOO stock latest news updates\"}"                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Risk Advisor                                                                                            │
│                                                                                                                 │
│  Thought: Thought: Since the tool encountered an error, I will attempt using another search query to identify   │
│  the key risks associated with trading VOO ETF, in case there was an issue with the specificity or formatting   │
│  of the original query.                                                                                         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"VOO ETF trading risks 2023\"}"                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url:  │
│  https://google.serper.dev/search.                                                                              │
│   Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper         │
│  Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the         │
│  internet', 'type': 'str'}}                                                                                     │
│  Tool Description: A tool that can be used to search the internet with a search_query. Supports different       │
│  search types: 'search' (default), 'news'.                                                                      │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [Search the internet with Serper, Read website content]           │
│  Action Input: the input to the action, dictionary enclosed in curly braces                                     │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│  This Thought/Action/Action Input/Result can repeat N times. Once I know the final answer, I must return the    │
│  following format:                                                                                              │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now can give a great answer                                                                         │
│  Final Answer: Your final answer must be the great and the most complete as possible, it must be outcome        │
│  described                                                                                                      │
│                                                                                                                 │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Risk Advisor                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The key risks associated with trading the Vanguard S&P 500 ETF (VOO) include:                                  │
│                                                                                                                 │
│  1. **Market Risk**: As VOO tracks the S&P 500, it is subject to fluctuations in the overall market. A          │
│  downturn in the U.S. stock market or a systemic decline across the industries represented in the index can     │
│  negatively impact the ETF's performance. Market volatility can arise from economic changes, political unrest,  │
│  changes in interest rates, or global economic conditions.                                                      │
│                                                                                                                 │
│  2. **Liquidity Risk**: While VOO is generally considered a highly liquid ETF due to its popularity and the     │
│  substantial daily trading volume, market conditions can impact its liquidity. For example, extreme market      │
│  volatility or disruptions in the trading systems could temporarily affect the ability to buy or sell the ETF   │
│  at preferred prices.                                                                                           │
│                                                                                                                 │
│  3. **Tracking Error**: This refers to the risk that the ETF’s performance may not perfectly match the          │
│  performance of the S&P 500 index it aims to replicate. Factors such as fund expenses, timing of rebalancing,   │
│  and differences in weighting methodology can lead to tracking errors.                                          │
│                                                                                                                 │
│  4. **Interest Rate Risk**: Changes in interest rates can affect the stock market, particularly stocks of       │
│  companies with high debt levels or those sensitive to interest rate changes, like those in the utilities and   │
│  real estate sectors. An increase in rates generally leads to lower stock prices, affecting the overall         │
│  performance of VOO.                                                                                            │
│                                                                                                                 │
│  5. **Regulatory Risks**: Changes in laws or regulations affecting the financial sector or securities markets,  │
│  including changes in tax policies and trade agreements, could impact the performance of the stock market and,  │
│  consequently, VOO.                                                                                             │
│                                                                                                                 │
│  Understanding these risks is crucial for anyone considering trading VOO or including it in their investment    │
│  portfolio, as they influence the overall risk-return profile of the investment.                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [19]:
# Convert to HTML for better formatting
from IPython.display import HTML
html_content = result.raw.replace('\n\n', '<br><br>').replace('\n', '<br>')
display(HTML(html_content))